In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

def get_treasure_links():
    base_url = "https://www.museum.go.kr/site/main/relic/treasure/list"
    all_data = []
    
    # 페이지 수는 실제 상황에 맞게 수정 (예: 37페이지)
    total_pages = 37
    
    for page in range(1, total_pages + 1):
        start_count = 12 * (page - 1)
        params = {"startCount": start_count}
        
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"페이지 {page} 접속 에러: status {response.status_code}")
            continue
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # li.card 안의 a.img-box를 찾아서 href를 추출
        cards = soup.select("li.card a.img-box")
        
        if not cards:
            print(f"페이지 {page}에 카드가 없습니다. 구조가 바뀌었을 수 있습니다.")
            continue
        
        for card in cards:
            detail_link = card.get("href", "")
            
            # 원하는 JSON 구조에 맞게 저장
            all_data.append({
                "page": page,
                "link": detail_link  # 예: "/site/main/relic/treasure/view?relicId=358"
            })
            print(f"페이지: {page}, 링크: {detail_link}")
        
        time.sleep(0.5)  # 서버 부하 방지용 임시 지연
    
    return all_data


data = get_treasure_links()
with open("treasure_links.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"수집 완료! 총 {len(data)}개의 링크를 저장했습니다.")

페이지: 1, 링크: /site/main/relic/treasure/view?relicId=348
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=349
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=350
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=355
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=358
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=372
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=482
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=487
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=499
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=517
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=518
페이지: 1, 링크: /site/main/relic/treasure/view?relicId=521
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=530
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=531
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=532
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=533
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=534
페이지: 2, 링크: /site/main/relic/treasure/view?relicId=535
페이지: 2, 링크

In [1]:
import os
import re
import json
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs

def parse_museum_page(url):
    """
    주어진 URL(국립중앙박물관 유물 상세)에서 
    - 대표(큰) 이미지
    - 썸네일 이미지 목록
    - 유물 정보(이름/국적/시대/재질/분류 등)
    - 본문 content
    등을 추출하여 하나의 JSON(dict)으로 반환
    """

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"페이지 응답 에러(HTTP {response.status_code}): {url}")
    soup = BeautifulSoup(response.text, "html.parser")

    # 결과를 담을 딕셔너리
    result = {
        "url": url,          # 어떤 URL을 파싱했는지
        "img": None,         # 대표 이미지
        "thumbs": [],        # 썸네일 목록
        "info": {},          # 상세 정보(제목 등)
        "content": ""        # 본문
    }

    # [썸네일 추출]
    thumb_slides = soup.select("div.swiper-container.gallery-thumbs a.swiper-slide")
    for tslide in thumb_slides:
        style_text = tslide.get("style", "")
        # style="background:url('/relic_image/xxx.jpg') ..."
        matches = re.findall(r'url\(["\'](.+?)["\']\)', style_text)
        if matches:
            result["thumbs"].append(matches[0])

    # [대표 이미지: 썸네일 중 첫 번째 사용]
    if result["thumbs"]:
        result["img"] = result["thumbs"][0]

    # [상세정보 파싱]  : outview-tit-box > outview
    outview_box = soup.select_one("div.outview-tit-box div.outview")
    if outview_box:
        # 제목
        title_el = outview_box.select_one("strong.outveiw-tit")
        if title_el:
            result["info"]["title"] = title_el.get_text(strip=True)

        # ul.outview-list
        info_list = outview_box.select("ul.outview-list > li")
        for li in info_list:
            strong_el = li.select_one("strong")
            p_el = li.select_one("p")
            if strong_el and p_el:
                key_text = strong_el.get_text(strip=True)
                val_text = p_el.get_text(strip=True)
                result["info"][key_text] = val_text

    # [본문 내용 파싱] : view-info-cont2 > p
    content_el = soup.select_one("div.view-info-cont.view-info-cont2 > p")
    if content_el:
        result["content"] = content_el.get_text(strip=True)

    return result

def get_relic_id_from_url(url: str) -> str:
    """
    URL의 쿼리스트링(querystring)에서 relicId=XXX 값을 추출.
    예: https://www.museum.go.kr/...relicId=36559443 -> '36559443'
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    # relicId 값이 존재하면 첫 번째 사용, 없으면 "unknown"
    relic_id = qs.get("relicId", ["unknown"])[0]
    return relic_id

def download_image(img_url: str, folder_path: str):
    """
    img_url을 다운로드해 folder_path 내에 저장.
    img_url이 절대경로(http...)가 아니면 
    https://www.museum.go.kr + img_url 로 가정.
    """
    if not img_url:
        return  # 값이 없으면 스킵

    if img_url.startswith("http"):
        full_url = img_url
    else:
        # 국립중앙박물관 사이트이므로 도메인 붙이기
        full_url = "https://www.museum.go.kr" + img_url

    try:
        r = requests.get(full_url, timeout=10)
        if r.status_code == 200:
            # url의 마지막 파일명 추출
            filename = os.path.basename(img_url)
            save_path = os.path.join(folder_path, filename)
            with open(save_path, "wb") as f:
                f.write(r.content)
            print(f" [이미지] 저장 완료: {save_path}")
        else:
            print(f" [이미지] 응답 에러: {full_url}, status={r.status_code}")
    except Exception as e:
        print(f" [이미지] 다운로드 예외: {full_url}, err={e}")

In [5]:
# treasure_links.json 파일을 읽어서 URL 리스트 생성
with open("treasure_links.json", "r", encoding="utf-8") as f:
    treasure_data = json.load(f)

# 기본 URL과 상세 링크를 결합하여 완전한 URL 리스트 생성
base_url = "https://www.museum.go.kr"
url_list = [base_url + item["link"] for item in treasure_data]

print(f"총 {len(url_list)}개의 URL이 로드되었습니다.")
print(f"첫 번째 URL 예시: {url_list[0]}")


총 433개의 URL이 로드되었습니다.
첫 번째 URL 예시: https://www.museum.go.kr/site/main/relic/treasure/view?relicId=348


In [6]:
# 1) "relics" 폴더를 만든다.
base_dir = "relics"
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    
for url in url_list:
    # relicId 추출
    relic_id = get_relic_id_from_url(url)
    print(f"\n=== {relic_id} 처리 중 ===")

    # 2) parse 함수로 데이터 추출
    data = parse_museum_page(url)

    # 3) relics/<relic_id> 폴더 생성
    relic_folder = os.path.join(base_dir, relic_id)
    if not os.path.exists(relic_folder):
        os.makedirs(relic_folder)

    # 4) json 파일로 저장
    json_path = os.path.join(relic_folder, "info.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f" [JSON] 저장 완료: {json_path}")

    # 5) 이미지 다운로드 (대표 이미지 + 썸네일)
    # 대표 이미지
    download_image(data["img"], relic_folder)
    # 썸네일 목록
    for thumb_url in data["thumbs"]:
        download_image(thumb_url, relic_folder)




=== 348 처리 중 ===
 [JSON] 저장 완료: relics\348\info.json
 [이미지] 저장 완료: relics\348\bon001958-000-0001.jpg
 [이미지] 저장 완료: relics\348\bon001958-000-0001.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-01.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-02.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-03.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-04.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-05.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-06.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-07.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-09.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-11.jpg
 [이미지] 저장 완료: relics\348\bon001958-00-08.jpg
 [이미지] 저장 완료: relics\348\bon001958-000-90000.jpg

=== 349 처리 중 ===
 [JSON] 저장 완료: relics\349\info.json
 [이미지] 저장 완료: relics\349\bon001959-00-01.jpg
 [이미지] 저장 완료: relics\349\bon001959-00-01.jpg
 [이미지] 저장 완료: relics\349\bon001959-00-02.jpg
 [이미지] 저장 완료: relics\349\bon001959-00-03.jpg
 [이미지] 저장 완료: relics\349\bon001959-00-04.jpg
 [이미지] 저장 완료: relics\349\bon001959-00-05.jpg
 [이미지] 저장 완료: relics\349\bo

In [11]:
# relics 폴더 내의 모든 유물 정보를 하나의 딕셔너리로 합치기
import os
import json

def combine_all_relics_info():
    combined_data = {}
    relics_dir = "relics"
    
    # relics 폴더 내의 모든 하위 폴더 확인
    for relic_id in os.listdir(relics_dir):
        relic_folder = os.path.join(relics_dir, relic_id)
        
        # 디렉토리인지 확인
        if os.path.isdir(relic_folder):
            json_path = os.path.join(relic_folder, "info.json")
            
            # info.json 파일이 존재하는지 확인
            if os.path.exists(json_path):
                try:
                    with open(json_path, "r", encoding="utf-8") as f:
                        relic_data = json.load(f)
                    
                    # relic_id를 숫자로 변환하여 키로 사용
                    numeric_id = int(relic_id)
                    combined_data[numeric_id] = relic_data
                    #print(f"Added relic {relic_id} to combined data")
                except ValueError as ve:
                    print(f"Error converting {relic_id} to integer: {ve}")
                except Exception as e:
                    print(f"Error processing {relic_id}: {e}")
    
    # 결과 저장
    output_path = "combined_relics.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(combined_data, f, ensure_ascii=False, indent=4)
    
    print(f"\n모든 유물 정보가 {output_path}에 저장되었습니다.")
    print(f"총 {len(combined_data)}개의 유물 정보가 결합되었습니다.")
    
    return combined_data

# 모든 유물 정보 합치기 실행
combined_relics = combine_all_relics_info()



모든 유물 정보가 combined_relics.json에 저장되었습니다.
총 433개의 유물 정보가 결합되었습니다.


In [14]:
sorted(list(combined_relics.keys()))

[348,
 349,
 350,
 355,
 358,
 372,
 482,
 487,
 499,
 517,
 518,
 521,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 618,
 630,
 638,
 655,
 662,
 691,
 693,
 694,
 703,
 706,
 715,
 752,
 767,
 768,
 856,
 857,
 858,
 941,
 958,
 964,
 967,
 974,
 1007,
 1086,
 1096,
 1126,
 1138,
 1149,
 1160,
 1166,
 1177,
 1193,
 1200,
 1204,
 1206,
 1210,
 1226,
 1227,
 1236,
 1256,
 1260,
 1261,
 1274,
 1296,
 1302,
 1343,
 1351,
 1376,
 1403,
 1425,
 1431,
 1434,
 1463,
 1464,
 1475,
 1476,
 1477,
 1484,
 1515,
 1734,
 1766,
 1768,
 1917,
 1921,
 1924,
 1933,
 1934,
 1939,
 1955,
 1969,
 1974,
 1978,
 1986,
 1991,
 2009,
 2012,
 2022,
 2023,
 2024,
 2025,
 2033,
 2048,
 2049,
 2074,
 2088,
 2089,
 2090,
 2094,
 2131,
 2132,
 2181,
 2195,
 2203,
 2224,
 2287,
 2288,
 2299,
 2300,
 2318,
 2327,
 2350,
 2361,
 2362,
 2371,
 2397,
 2399,
 2430,
 2480,
 2481,
 2483,
 2489,
 2549,
 2577,
 3150,
 3151,
 

In [18]:
combined_relics[499]['info']
combined_relics[499]['content']

"추사(秋史) 김정희(金正喜, 1786-1856)의 대표적인 작품이다. '묵소거사(默笑居士)'에 맞춰 지은 상징적인 어구로, '침묵을 지켜야 할 때는 그 때에 맞게, 또한 웃어야 할 때는 웃어야 할 때에 맞게' 라는 구절을 시작으로 삶의 깨달음을 풀어놓은 듯한 내용으로 전개하였다. 한편 작품을 감싼 장황 주변에 찍힌 엽전 모양의 둥근 인장들은 그의 친한 친구였던 김유근(金?根, 1785-1840)의 것으로, 이 <묵소거사자찬>이 김유근과 관계 있음을 보여준다. 정중하게 정성을 담아 쓴 듯한 이 글씨는 글씨형이 세장(細長)하며, 날카롭고 변화가 큰 필획을 구사하였다."

In [32]:
# 이미지 URL에서 파일명 추출
img_url = combined_relics[348]['img']
# 이미지 URL에서 파일명 추출
img_filename = img_url.split('/')[-1]
print(f"Image filename: {img_filename}")

# 전체 파일 경로 생성
file_path = os.path.join("relics", "348", img_filename)
print(f"Full file path: {file_path}")


# 전체 데이터도 출력
combined_relics[348]

Image filename: bon001958-000-0001.jpg
Full file path: relics\348\bon001958-000-0001.jpg


{'url': 'https://www.museum.go.kr/site/main/relic/treasure/view?relicId=348',
 'img': '/relic_image/PS01001001/bon001/2016/1124093013018/700/bon001958-000-0001.jpg',
 'thumbs': ['/relic_image/PS01001001/bon001/2016/1124093013018/700/bon001958-000-0001.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-01.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-02.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-03.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-04.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-05.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-06.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-07.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-09.jpg',
  '/relic_image/PS01001001/bon001/2017/0508160735009/700/bon001958-00-11.jpg',
  '/relic_image/PS01001001/bon0

In [103]:
from IPython.display import Image
#Image(file_path)

In [93]:
combined_relics[348]['info']
combined_relics[348]['content']

{"설명": combined_relics[348]['content'], "세부정보": combined_relics[348]['info']}

{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.',
 '세부정보': {'title': '감산사 석조미륵보살입상',
  '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像',
  '전시명칭': '감산사미륵보살과 아미타불',
  '국적/시대': '한국 - 통일신라',
  '출토지': '출토지 - 경상북도',
  '재질': '돌 - 화강암',
  '분류': '종교신앙 - 불교 - 예배 - 불상',
  '크기': '높이 270.0cm',
  '지정문화유산': '국보',
  '소장품번호': '본관 1958',
  '전시위치': '불교조각'}}

In [53]:
f"{data}\n\n(1)이미지를 설명해주세요 (2)이미지의 당신만의 감상을 파세요. (3) 이미지의 어떤 감정일 때 보면 좋을 까ㅛ?"

"{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.', '세부정보': {'title': '감산사 석조미륵보살입상', '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像', '전시명칭': '감산사미륵보살과 아미타불', '국적/시대': '한국 - 통일신라', '출토지': '출토지 - 경상북도', '재질': '돌 - 화강암', '분류': '종교신앙 - 불교 - 예배 - 불상', '크기': '높이 270.0cm', '지정문화유산': '국보', '소장품번호': '본관 1958', '전시위치': '불교조각'}}\n\n(1)이미지를 설명해주세요 (2)이미지의 당신만의 감상을 파세요. (3) 이미지의 어떤 감정일 때 보면 좋을 까ㅛ?"

In [54]:
data

{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.',
 '세부정보': {'title': '감산사 석조미륵보살입상',
  '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像',
  '전시명칭': '감산사미륵보살과 아미타불',
  '국적/시대': '한국 - 통일신라',
  '출토지': '출토지 - 경상북도',
  '재질': '돌 - 화강암',
  '분류': '종교신앙 - 불교 - 예배 - 불상',
  '크기': '높이 270.0cm',
  '지정문화유산': '국보',
  '소장품번호': '본관 1958',
  '전시위치': '불교조각'}}

In [68]:
data

{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.',
 '세부정보': {'title': '감산사 석조미륵보살입상',
  '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像',
  '전시명칭': '감산사미륵보살과 아미타불',
  '국적/시대': '한국 - 통일신라',
  '출토지': '출토지 - 경상북도',
  '재질': '돌 - 화강암',
  '분류': '종교신앙 - 불교 - 예배 - 불상',
  '크기': '높이 270.0cm',
  '지정문화유산': '국보',
  '소장품번호': '본관 1958',
  '전시위치': '불교조각'}}

In [94]:
information = str({"설명": combined_relics[348]['content'], "세부정보": combined_relics[348]['info']})

In [104]:
user_message = """
당신은 한국의 국립중앙박물관 도슨트입니다. 제공된 유물 이미지를 보고 감상평을 남겨야 합니다. 유물의 세부적이 내용은 박물관에서 제공하는 <information>를 참고하세요.

<information>
{information}
</information>

<instructions>
1. <json> 태그로 감싸 다음 JSON 포맷으로 응답하세요:
{{"저의 느낌입니다.":<5문장 이내로>, "일상애세 이런 감정인 사람에게 보여주고 싶어요.":<5문장 이내로>, "너의 감상을 5개의 키워드로 표현해":[...]}}
</instructions>
""".format(information=information)

print(user_message)


당신은 한국의 국립중앙박물관 도슨트입니다. 제공된 유물 이미지를 보고 감상평을 남겨야 합니다. 유물의 세부적이 내용은 박물관에서 제공하는 <information>를 참고하세요.

<information>
{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.', '세부정보': {'title': '감산사 석조미륵보살입상', '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像', '전시명칭': '감산사미륵보살과 아미타불', '국적/시대': '한국 - 통일신라', '출토지': '출토지 - 경상북도', '재질': '돌 - 화강암', '분류': '종교신앙 - 불교 - 예배 - 불상', '크기': '높이 270.0cm', '지정문화유산': '국보', '소장품번호': '본관 1958', '전시위치': '불교조각'}}
</information>

<instructions>
1. <json> 태그로 감싸 다음 JSON 포맷으로 응답하세요:
{"저의 느낌입니다.":<5문장 이내로>, "일상애세 이런 감정인 사람에게 보여주고 싶어요.":<

In [105]:
import anthropic
import base64

# 로컬 파일을 base64로 인코딩
with open(file_path, "rb") as f:
    base64_data = base64.standard_b64encode(f.read()).decode("utf-8")

client = anthropic.Anthropic()
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1024,
    temperature=0,
    stop_sequences=["</json>"],
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_data
                    }
                },
                {
                    "type": "text", 
                    #"text": f"{data}\n\n(1)이미지를 설명해주세요 (2)이미지의 당신만의 감상을 파세요. (3) 이미지의 어떤 감정일 때 보면 좋을 까? 한국어로 답할 것"
                    "text": user_message
                }
            ]
        },
        {"role": "assistant", "content": "<json>"}
    ],
)

In [102]:
print(message.content[0].text)


{
"저의 느낌입니다.": "이 미륵보살상의 우아한 자태와 섬세한 조각 기법에 감탄합니다. 8세기 통일신라 시대의 뛰어난 조각 기술이 돋보이며, 특히 얼굴의 미소와 신체의 유려한 곡선이 평온함을 전달합니다. 김지성이 어머니를 위해 조성했다는 사연이 더욱 이 작품에 깊은 의미를 부여합니다. 화려한 장식과 사실적인 표현이 당시 불교 예술의 정수를 보여줍니다.",
"일상애세 이런 감정인 사람에게 보여주고 싶어요.": "고요함과 평화를 추구하는 사람에게 이 미륵보살상을 보여주고 싶습니다. 어머니를 그리워하는 마음을 가진 이들에게 위로가 될 것 같습니다. 예술과 역사에 관심이 많은 친구들에게 통일신라 불교 예술의 아름다움을 나누고 싶습니다. 삶의 균형과 조화를 찾는 이들에게 영감을 줄 수 있을 것입니다.",
"너의 감상을 5개의 키워드로 표현해": ["우아함", "모성애", "장인정신", "평온함", "시대의 정수"]
}



In [72]:
data

{'설명': '신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.',
 '세부정보': {'title': '감산사 석조미륵보살입상',
  '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像',
  '전시명칭': '감산사미륵보살과 아미타불',
  '국적/시대': '한국 - 통일신라',
  '출토지': '출토지 - 경상북도',
  '재질': '돌 - 화강암',
  '분류': '종교신앙 - 불교 - 예배 - 불상',
  '크기': '높이 270.0cm',
  '지정문화유산': '국보',
  '소장품번호': '본관 1958',
  '전시위치': '불교조각'}}